In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

import sys

'''
Combined 2 rows into 1 by taking the previous one.
Made sure that both rows are from the same users.

User_ID1,Doc_ID1,Inter-Stroke time1,Stroke Duration1,Start X1,Start Y1,Stop X1,Stop Y1,Direct end-to-end Distance1,
Mean resulant length1,WSAD Flag1,Direction of end-to-end Time1,Phone_ID1,20 per pairwise velocity1,50 per pairwise velocity1,
80 per pairwise velocity1,20 per pairwise acc1,50 per pairwise acc1,80 per pairwise acc1,Median velocity at last 3pts1,
Largest deviation from end-to-end line1,20 per dev line1,50 per dev line1,80 per dev line1,Average Direction1,
Length of Trajectory1,Ratio dist and length of trajectory1,Average velocity1,Median acceleration at first 5 pts1,
Mid-Stroke pressure1,Mid-Stroke area covered1,Mid-Stroke finger orientation1,Change of finger orientation1,
Phone orientation1

User_ID2,Doc_ID2,Inter-Stroke time2,Stroke Duration2,Start X2,Start Y2,Stop X2,Stop Y2,Direct end-to-end Distance2,
Mean resulant length2,WSAD Flag2,Direction of end-to-end Time2,Phone_ID2,20 per pairwise velocity2,50 per pairwise velocity2,
80 per pairwise velocity2,20 per pairwise acc2,50 per pairwise acc2,80 per pairwise acc2,Median velocity at last 3pts2,
Largest deviation from end-to-end line2,20 per dev line2,50 per dev line2,80 per dev line2,Average Direction2,
Length of Trajectory2,Ratio dist and length of trajectory2,Average velocity2,Median acceleration at first 5 pts2,
Mid-Stroke pressure2,Mid-Stroke area covered2,Mid-Stroke finger orientation2,Change of finger orientation2,
Phone orientation2
'''

import os
#print(os.listdir(r"C:\Users\eddie\OneDrive\Desktop\CISC 4900"))         #print out what's in the file/directory
#train = pd.read_csv(r"C:\Users\eddie\OneDrive\Desktop\CISC 4900\Dataset_V4.csv")         #using the combined dataset

print(os.listdir(r"."))         #print out what's in the file/directory
train = pd.read_csv(r"Dataset_V4.csv")         #using test

['CISC 4900 First Interim Report - Eddie Lam.pdf', 'CombinedRows.csv', 'DataSet_V4.csv', 'EditingDataset_V3.java', 'featMat.csv', 'featMatVersion2_10.csv', 'featMatVersion2_5.csv', 'featMatVersion2_6.csv', 'featMatVersion2_7.csv', 'featMatVersion2_8.csv', 'featMatVersion2_9.csv', 'features', 'Old dataset', 'Test_dataset.csv', 'Touchalytics-ProjectDef.pdf', 'Train_dataset.csv']


In [2]:
#prints the amount of users in 'User_ID'
train['User_ID1'].value_counts()

2     1229
35    1062
23     968
38     785
3      758
15     718
33     711
28     709
17     705
21     701
14     650
8      617
34     612
27     608
7      590
6      561
36     499
18     460
9      452
11     444
29     438
39     435
24     434
25     431
37     423
5      412
1      400
10     386
16     381
22     373
31     369
20     355
12     341
40     311
13     304
32     303
19     292
26     241
4      240
30     224
41     185
Name: User_ID1, dtype: int64

In [4]:
#Choosing the columns for X
#Not using 'User_ID', 'Doc_ID', or 'Phone_ID'
X = train[['Inter-Stroke time1', 'Stroke Duration1', 'Start X1', 'Start Y1', 'Stop X1', 'Stop Y1', 'Mean resulant length1',
           'WSAD Flag1', 'Direction of end-to-end Time1', '20 per pairwise velocity1', '50 per pairwise velocity1', 
           '80 per pairwise velocity1', '50 per pairwise acc1', '80 per pairwise acc1',
           'Median velocity at last 3pts1', '20 per dev line1', '50 per dev line1', '80 per dev line1', 
           'Length of Trajectory1', 'Ratio dist and length of trajectory1', 'Average velocity1', 'Mid-Stroke pressure1', 
           'Mid-Stroke area covered1', 'Change of finger orientation1', 'Phone orientation1',
           
           'Inter-Stroke time2', 'Stroke Duration2', 'Start X2', 'Start Y2', 'Stop X2', 'Stop Y2', 'Mean resulant length2', 
           'WSAD Flag2', 'Direction of end-to-end Time2', '20 per pairwise velocity2', '50 per pairwise velocity2', 
           '80 per pairwise velocity2', '50 per pairwise acc2', '80 per pairwise acc2', 
           'Median velocity at last 3pts2', '20 per dev line2', '50 per dev line2', '80 per dev line2', 
           'Length of Trajectory2', 'Ratio dist and length of trajectory2', 'Average velocity2', 'Mid-Stroke pressure2', 
           'Mid-Stroke area covered2', 'Change of finger orientation2', 'Phone orientation2']].values

#can't use some of them
'''
 'Direct end-to-end Distance1', 'Largest deviation from end-to-end line1', 'Average Direction1', 
 'Median acceleration at first 5 pts1', 'Mid-Stroke finger orientation1', '20 per pairwise acc1', 
 
 'Direct end-to-end Distance2', 'Largest deviation from end-to-end line2', 'Average Direction2', 
 'Median acceleration at first 5 pts2', 'Mid-Stroke finger orientation2', '20 per pairwise acc2', 
'''

"\n 'Direct end-to-end Distance1', 'Largest deviation from end-to-end line1', 'Average Direction1', \n 'Median acceleration at first 5 pts1', 'Mid-Stroke finger orientation1', '20 per pairwise acc1', \n \n 'Direct end-to-end Distance2', 'Largest deviation from end-to-end line2', 'Average Direction2', \n 'Median acceleration at first 5 pts2', 'Mid-Stroke finger orientation2', '20 per pairwise acc2', \n"

In [5]:
#Choosing the lables for y
y = train['User_ID1'].values

In [6]:
#Train and test split
#80% train, 20% test
#splitting the data into test and train dataset

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [7]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
#Training and Prediction
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)         #value for k
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [9]:
#Make prediction on the test data
y_pred = classifier.predict(X_test)

In [10]:
#Evaluating the Algorithm
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 56   0   0 ...   0   0   2]
 [  1 190   1 ...   0   1   0]
 [  0   2  67 ...   0   0   0]
 ...
 [  1   1   3 ...  38   0   0]
 [  0   0   0 ...   0  36   0]
 [  2   2   0 ...   0   0  30]]
              precision    recall  f1-score   support

           1       0.59      0.70      0.64        80
           2       0.69      0.90      0.78       212
           3       0.48      0.44      0.46       152
           4       0.68      0.69      0.69        52
           5       0.39      0.44      0.41        77
           6       0.61      0.72      0.66       112
           7       0.61      0.58      0.59       112
           8       0.53      0.66      0.59       114
           9       0.76      0.86      0.81        90
          10       0.65      0.62      0.64        82
          11       0.72      0.73      0.72        81
          12       0.67      0.82      0.73        66
          13       0.55      0.52      0.53        60
          14       0.62      0.87      0.72       1

In [11]:
#prints the entire confusion matrix
#however, it doesnn't fit the entire row and loops around
np.set_printoptions(threshold=sys.maxsize)
resultCM = (confusion_matrix(y_test, y_pred))
print(resultCM)

[[ 56   0   0   4   0   0   2   0   0   0   0   1   0   0   2   0   0   0
    0   0   0   3   0   1   4   1   0   1   0   0   0   0   0   1   0   1
    0   1   0   0   2]
 [  1 190   1   0   0   0   0   0   1   0   0   0   1   0   0   0   0   0
    0   0   0   0   4   1   0   0   1   2   1   0   0   0   0   1   0   1
    0   6   0   1   0]
 [  0   2  67   0   4   4   2   5   1   5   1   2   0   0   0   4  22   3
    0   0   6   0   0   1   0   0   2   0   0   0   0   0   0   0  14   0
    4   3   0   0   0]
 [  7   0   0  36   0   0   0   0   0   0   0   0   0   1   0   0   0   0
    0   1   0   1   0   0   0   1   0   3   0   0   0   0   0   2   0   0
    0   0   0   0   0]
 [  1   0   8   0  34   3   2   3   5   0   0   1   1   0   0   1   0   1
    5   0   0   0   0   1   0   0   3   0   0   0   0   3   1   0   0   0
    4   0   0   0   0]
 [  0   3   4   0   2  81   0   3   0   2   0   0   2   0   0   6   3   0
    0   0   0   0   0   1   0   0   0   1   0   0   0   0   0   0   3  

In [12]:
resultCR = (classification_report(y_test, y_pred))
print(resultCR)

              precision    recall  f1-score   support

           1       0.59      0.70      0.64        80
           2       0.69      0.90      0.78       212
           3       0.48      0.44      0.46       152
           4       0.68      0.69      0.69        52
           5       0.39      0.44      0.41        77
           6       0.61      0.72      0.66       112
           7       0.61      0.58      0.59       112
           8       0.53      0.66      0.59       114
           9       0.76      0.86      0.81        90
          10       0.65      0.62      0.64        82
          11       0.72      0.73      0.72        81
          12       0.67      0.82      0.73        66
          13       0.55      0.52      0.53        60
          14       0.62      0.87      0.72       134
          15       0.57      0.70      0.63       128
          16       0.50      0.53      0.51        81
          17       0.66      0.67      0.66       144
          18       0.82    